# Examen ETL: SPARK 02/02

Se podrá utilizar toda la información que se encuentra en el campus. 

El fichero de datos sobre el que se trabajará es el de partidosLigaNBA.txt.

A cada una de las preguntas hay que responder explicando brevemente que se pretende hacer antes de lanzar el código.

Al documento lo llamareís con vuestro nombre y apellido. Debeís enviarlo a mi correo de CUNEF antes del final del examen.

El lenguaje para trabajar con Spark podrá ser python o R indistintamente.

## Primera pregunta: Describe brevemente que diferencia el persists, cache y collect en spark. Explica brevemente casos en los que es interesante su aplicación

# Respuesta 1:

El persist sirve para que cuando haces transformaciones sobre un RDD y luego una acción, con el que obtendrás una salida en python.
El persist hace que se te guarde el paso intermedio, ahorrando tiempo de ejecucion la proxima vez que hagas una llamada a ese objeto.
Por ejemplo:





Con datos pequeños usamos cache, que lo guarda en memoria. El persist hace lo mismo, sólo que puedes elegir si lo guarda en memoria o en disco, (generalmente lo hará en disco), pudiendos incluso guardar una parte en una y otra en otra en el caso de que el dato sea demasiado pesado.
 
El collect por su parte, es una acción que te trae el RDD y tiene como output un objeto en el lenguaje que estés utilizando,
por ejemplo en Pyspark será un objeto tipo lista.

## Segunda pregunta: Explica brevemente los pasos a seguir para realizar la carga de un conjunto de datos (pasos que se siguieron en la práctica con datos de logs)

# Respuesta 2:


- En primer lugar habrá que definir la fuente o fuentes de los que extraeremos los datos. Estas pueden ser una API, un csv, una URL, HDFS...
- Una vez tengamos esto, deberemos establecer una conexión con la fuente, que variará obviamente según el tipo de esta.
- En tercer lugar deberemos extraer el dato. Este puede estar en diferentes formatos, sólo a título de ejemplo nombrar csv, Apache Parquet, GZ, txt...
- Una vez que tenemos el dato, deberemos cargarlo en Spark. Para ello a su vez deberemos definir y cargar (ya sea en local o en distribuido) el Spark Context en el entorno en el que estemos trabajando. En PySpark por ejemplo se haría con un parallelize()
- Luego debemos comprobar que la carga se ha realizado correctamente, por ejemplo realizando una accion de take() o un first(). Tambien se debe muestrear con un takeSampe o un sample() + collect() Con esto vemos la estructura de los datos como se ha cargado.
- Detectar el formato con los separadores y el correspondiente parseado de datos y fechas. Aquí deberemos usar funciones de parseado y en muchas ocasiones expresiones regulares para leer los datos de la manera correcta que posteriormente nos permita y nos facilite el trabajo. Comprobamos que las frecuencias, las separaciones y los metadatos estén correctamente.

EL PASO A PASO QUEDARÍA:

* cargar datos
* extraer una muestra
* hacer parseo
* tipos de datos
* cargar todos los datos
* distribuirlos si el espacio no es suficiente
* mismas columnas que el fichero original
* validar dominio (estructura de los datos) summary
* analisis descriptivo
* generamos variables
* estudiar el balance de la variable objetivo
* dividimos en train test
* creamos modelo
* testeamos el modelo

## Tercera Pregunta: Índica un tipo de problema que puede empeorar los datos. (pe. Que no exista un representante del CDO en todas las áreas de negocio), pon algún ejemplo específico (pe. Datos duplicados) y cómo lo tratarías con técnicas de data cleaning.

# Respuesta 3:


El primer ejemplo que se me viene a la mente, como problema en una fuente de datos, ocurre en los formularios online. Cada vez cobra más importancia este tipo de fuente de datos, un buen ejemplo es el caso de la banca que ya permite adquirir una gran parte de sus productos en un proceso totalmente online. Para obtener los datos totalmente imprescindibles para dar de alta a estos clientes, necesitan unos formularios infalibles que proporcionen unos datos de buena calidad. 
Sin embargo, estos formularios plantean numerosos problemas como puede ser el caso de los campos de edad y numero de telefono. El primero de ellso suele traer datos por defecto, lo cual deberemos detectar en nuestro proceso de Data Cleaning. En cuanto a los números de telefono, la gente que no quiere darlos, al ser un campo normalmente obligatoorio tiende a inverntarselo. Para que este hecho noperjudique nuestro análisis, podemos detectar los valores únicos más repetidos, que serán estos números de teléfono inventados. Pero es más,  incluso podríamos extraer valor de estos datos creando una nueva variable que nos diga si el cliente ha proporcionado o no su número de telefono. Esto nos podría indicar fidelidad o no hacia la marca, confianza transmitida por la marca o la página web....

Por ejemplo también se me puede ocurrir tablas intermedias desactualizadas que cuando las juntas tienes el dato repetido y diferente. En este caso habría que ver, por conocimiento de negocio o algun metadato que indique la frecuencia de actualización, a que tabla dar credibilidad.

Otro de los problemas peores que se me ocurran es por ejemplo, y relacionado con lo primero, un dataset donde no existan NAs, sino que los NAs sean por ejemplo un 1 al ser una tasa de variación y no tener información de este año y repetir la del año anterior.

Otro ejemplo podría ser una serie temporal con valores ausentes que hay que decidir que hacer con ellos. Desde series financieras que no coticen los fines de semana a estadísticas de Google Analytics por caida de pagina web. Relacionado con esto, los malditos bots rusos también pueden distorsionar las métricas webs aumentando la tasa de abandono y contabilizando visitas que te distorsionen el seguimiento de por ejemplo una campaña. Para todo ello habría que unir el conocimiento experto con una serie de filtros, contadores de frecuencias... para solucionar el problema.



## Cuarta tarea: Inicializar spark context y cargar los datos desde el fichero.

In [27]:
# Creo la funcion para parsear las fechas
import datetime
import time

def formatoFecha(elemento):

    elemento=time.strptime(elemento, "%a, %b %d, %Y")

    return datetime.datetime(elemento[0],elemento[1],elemento[2],elemento[3])


In [181]:
# Probamos que funciona bien
a="Tue, Oct 30, 2007"
formatoFecha(a)

datetime.datetime(2007, 10, 30, 0, 0)

In [182]:
formatoFecha(a).month

10

In [183]:
# Creamos la funcion para parsear el RDD:
from pyspark.sql import Row
import datetime

#  ESTA FUNCION RECIBE UN RRD SIN HEADER
def parseaditoBomba(lineas):
    
    """SPLITEAMOS POR ':'"""
    elementos = lineas.split(":")  
    
    """AL HABER HECHO EL ANTERIOR SPLIT, NO SEPARA LA HORA Y LOS MINUTOS POR LO QUE LOS JUNTAMOS EN EL ELEMENTO 1"""
    """AL SER STRING LA FROMA MAS FACIL ES CON ''+' """
    elementos[1]= elementos[1]+":"+elementos[2]  
    
    """ELIMINAMOS EL ELEMENTO 2 QUE YA ESTÁ AÑADIDO AL ELEMENTO 1"""
    del elementos[2]
    
    """DEVOLVEMOS LA LINEA PONIENDOLE EL METADATO DEL NOMBRE DE CADA ELEMENTO Y EL FORMATO CORRECTO"""
    return(Row(
        Date=formatoFecha(elementos[0]),
        Dstart = elementos[1],  
        Visitor= elementos[2],
        PTSvis  = int(elementos[3]),
        Home    = elementos[4],
        PTShom  = int(elementos[5])
    ))


# La expresion regular a usar seria (.+):"(\d*:\d*\s\w*)":(\w+\s*\w*\s*\w*):(\d*):(\w+\s*\w*\s*\w*):(\d*)

In [1]:
# Inicializamos el Spark Context
from pyspark import SparkContext
sc = SparkContext("local","Simple App")

In [61]:
# Cargamos los datos desde el fichero
data_file = "./partidosLigaNBAClean.csv"
partidos = sc.textFile(data_file)

In [62]:
# Primero quitamos el header para pasarle el output correcto a la funcion creada
header = partidos.first()
partidos = partidos.filter(lambda x : x != header)

In [63]:
# Aplicamos el parseado y vemos que todo está correcto
partidos_sep = partidos.map(parseaditoBomba)
partidos_sep.take(5)

[Row(Date=datetime.datetime(2007, 10, 30, 0, 0), Dstart='"7:30 pm"', Home='Golden State Warriors', PTShom=96, PTSvis=117, Visitor='Utah Jazz'),
 Row(Date=datetime.datetime(2007, 10, 30, 0, 0), Dstart='"7:30 pm"', Home='Los Angeles Lakers', PTShom=93, PTSvis=95, Visitor='Houston Rockets'),
 Row(Date=datetime.datetime(2007, 10, 30, 0, 0), Dstart='"7:00 pm"', Home='San Antonio Spurs', PTShom=106, PTSvis=97, Visitor='Portland Trail Blazers'),
 Row(Date=datetime.datetime(2007, 10, 31, 0, 0), Dstart='"8:00 pm"', Home='Cleveland Cavaliers', PTShom=74, PTSvis=92, Visitor='Dallas Mavericks'),
 Row(Date=datetime.datetime(2007, 10, 31, 0, 0), Dstart='"8:30 pm"', Home='Denver Nuggets', PTShom=120, PTSvis=103, Visitor='Seattle SuperSonics')]

## Quinta tarea: Media de la diferencia de puntos por año

In [79]:
# 1º En primer lugar mapeamos poniedo como clave el año, y como valor la diferencia de puntos y un contador, por si el numero de 
# partidos por temporada no fuera el mismo:
años_diferencias = partidos_sep.map(lambda x: (x[0].year, (abs(x[3]-x[4]),1)))
años_diferencias.take(5)

[(2007, (21, 1)),
 (2007, (2, 1)),
 (2007, (9, 1)),
 (2007, (18, 1)),
 (2007, (17, 1))]

In [81]:
# 2º Una vez tenemos lo anterior, agrupamos sumando la diferencia y el numero de partidos por año.
años_agrupado = años_diferencias.reduceByKey(lambda x,y: (x[0]+y[0], x[1]+y[1]))
años_agrupado.take(5)

[(2016, (15397, 1333)),
 (2017, (9172, 803)),
 (2007, (5060, 456)),
 (2008, (15376, 1332)),
 (2009, (14595, 1316))]

### Respuesta :

In [83]:
# 3º Mapeamos para quedarnos con los años y realizar la media de las diferencias. 
# Como podemos ver, todo se ha realizado usando transformaciones hasta este punto que hacemos un collect() para obtere los resultados
años_agrupado.map(lambda x: (x[0], x[1][0]/x[1][1])).collect()

[(2016, 11.550637659414853),
 (2017, 11.422166874221668),
 (2007, 11.096491228070175),
 (2008, 11.543543543543544),
 (2009, 11.090425531914894),
 (2010, 10.86903860711582),
 (2011, 10.661016949152541),
 (2012, 10.845318860244234),
 (2013, 11.071752265861027),
 (2014, 10.9047976011994),
 (2015, 11.159969673995452)]

In [197]:
# Podríamos ordenarlo por año o por media más alta. Elegimos lo segundo:
años_agrupado.map(lambda x: (x[0], x[1][0]/x[1][1])).sortBy(lambda x : x[1], ascending= False).collect()

[(2016, 11.550637659414853),
 (2008, 11.543543543543544),
 (2017, 11.422166874221668),
 (2015, 11.159969673995452),
 (2007, 11.096491228070175),
 (2009, 11.090425531914894),
 (2013, 11.071752265861027),
 (2014, 10.9047976011994),
 (2010, 10.86903860711582),
 (2012, 10.845318860244234),
 (2011, 10.661016949152541)]

## Sexta tarea: ¿Han judado todos los equipos el mismo número de partidos? ¿ Si es qué no a que puede deberse?

In [172]:
# 1º En primer lugar mapeamos para fijar como clave el equipo y la temporada.
partidos_team_temporada = partidos_sep.map(lambda x: ((x[0].year,x[2]), 1)).reduceByKey(lambda x,y: x+y)
partidos_team_temporada.take(15)

[((2008, 'Washington Wizards'), 45),
 ((2007, 'New York Knicks'), 17),
 ((2008, 'New York Knicks'), 38),
 ((2016, 'Golden State Warriors'), 56),
 ((2017, 'Detroit Pistons'), 24),
 ((2014, 'Detroit Pistons'), 40),
 ((2012, 'Dallas Mavericks'), 46),
 ((2010, 'Minnesota Timberwolves'), 38),
 ((2010, 'Boston Celtics'), 54),
 ((2016, 'Charlotte Hornets'), 43),
 ((2013, 'Oklahoma City Thunder'), 45),
 ((2008, 'Atlanta Hawks'), 44),
 ((2012, 'Houston Rockets'), 49),
 ((2010, 'Denver Nuggets'), 45),
 ((2017, 'Sacramento Kings'), 27)]

In [175]:
# 2º Elegimos un año cualquiera para comprobar como efectivamete ocurre este hecho:
partidos_team_temporada.filter(lambda x : x[0][0] == 2017).collect()

[((2017, 'Detroit Pistons'), 24),
 ((2017, 'Sacramento Kings'), 27),
 ((2017, 'Minnesota Timberwolves'), 24),
 ((2017, 'Los Angeles Clippers'), 28),
 ((2017, 'Toronto Raptors'), 30),
 ((2017, 'Indiana Pacers'), 26),
 ((2017, 'Phoenix Suns'), 26),
 ((2017, 'Brooklyn Nets'), 26),
 ((2017, 'Oklahoma City Thunder'), 24),
 ((2017, 'San Antonio Spurs'), 34),
 ((2017, 'New Orleans Pelicans'), 21),
 ((2017, 'Charlotte Hornets'), 23),
 ((2017, 'Philadelphia 76ers'), 22),
 ((2017, 'Boston Celtics'), 37),
 ((2017, 'Golden State Warriors'), 34),
 ((2017, 'Houston Rockets'), 31),
 ((2017, 'Memphis Grizzlies'), 24),
 ((2017, 'Dallas Mavericks'), 26),
 ((2017, 'Atlanta Hawks'), 28),
 ((2017, 'New York Knicks'), 25),
 ((2017, 'Cleveland Cavaliers'), 30),
 ((2017, 'Milwaukee Bucks'), 27),
 ((2017, 'Denver Nuggets'), 25),
 ((2017, 'Washington Wizards'), 28),
 ((2017, 'Portland Trail Blazers'), 27),
 ((2017, 'Utah Jazz'), 27),
 ((2017, 'Los Angeles Lakers'), 25),
 ((2017, 'Miami Heat'), 24),
 ((2017, 'Or

### Respuesta:


Como podemos comprobar, los algunos equipos tienen más partidos que otros debido (además de que estamos tomando los datos por año y no por temporada) a que **la temporadas se divide en dos fases, la temporada regular y fase de play-off**. Vemos como los Warriors son los que más partidos tienne al haber llegado a la final del anillo. Los `Cavaliers` también deberían aparecer con el mismo número de partidos, pero por el hecho de estar tratando los datos por año y no por temporada podría explicar este hecho.

## Séptima pregunta: ¿Cuantos partidos ha ganado en Enero(todos los eneros) Cleveland Cavaliers?

Si no supiesemos el nombre exacto del equipo, podríamos hacer alguna busqueda con el módulo "re" para asegurarnos de que 
 sabemos el nombre correcto del equipo, algo como:

import re

re.search("Cleveland", x)

In [88]:
# 1º En primer lugar filtramos para obtener todos los partidos de los Cavs:
partidos_cavs = partidos_sep.filter(lambda x : x[2] == "Cleveland Cavaliers" or x[5] == "Cleveland Cavaliers")
partidos_cavs.take(5)

[Row(Date=datetime.datetime(2007, 10, 31, 0, 0), Dstart='"8:00 pm"', Home='Cleveland Cavaliers', PTShom=74, PTSvis=92, Visitor='Dallas Mavericks'),
 Row(Date=datetime.datetime(2007, 11, 2, 0, 0), Dstart='"7:30 pm"', Home='Cleveland Cavaliers', PTShom=110, PTSvis=106, Visitor='New York Knicks'),
 Row(Date=datetime.datetime(2007, 11, 4, 0, 0), Dstart='"6:00 pm"', Home='Phoenix Suns', PTShom=103, PTSvis=92, Visitor='Cleveland Cavaliers'),
 Row(Date=datetime.datetime(2007, 11, 6, 0, 0), Dstart='"7:30 pm"', Home='Golden State Warriors', PTShom=104, PTSvis=108, Visitor='Cleveland Cavaliers'),
 Row(Date=datetime.datetime(2007, 11, 7, 0, 0), Dstart='"8:30 pm"', Home='Utah Jazz', PTShom=103, PTSvis=101, Visitor='Cleveland Cavaliers')]

In [96]:
# 2º En segundo lugar, una vez tenemos los partidos de los Cavs, filtramos para obtener los que se jugaron en el mes 1(Enero):
partidos_cavs_enero = partidos_cavs.filter(lambda x: x[0].month == 1)
partidos_cavs_enero.take(5)

[Row(Date=datetime.datetime(2008, 1, 2, 0, 0), Dstart='"7:00 pm"', Home='Cleveland Cavaliers', PTShom=98, PTSvis=94, Visitor='Atlanta Hawks'),
 Row(Date=datetime.datetime(2008, 1, 4, 0, 0), Dstart='"7:30 pm"', Home='Cleveland Cavaliers', PTShom=97, PTSvis=93, Visitor='Sacramento Kings'),
 Row(Date=datetime.datetime(2008, 1, 6, 0, 0), Dstart='"1:00 pm"', Home='Toronto Raptors', PTShom=90, PTSvis=93, Visitor='Cleveland Cavaliers'),
 Row(Date=datetime.datetime(2008, 1, 8, 0, 0), Dstart='"7:00 pm"', Home='Cleveland Cavaliers', PTShom=95, PTSvis=79, Visitor='Seattle SuperSonics'),
 Row(Date=datetime.datetime(2008, 1, 9, 0, 0), Dstart='"7:00 pm"', Home='Atlanta Hawks', PTShom=90, PTSvis=81, Visitor='Cleveland Cavaliers')]

In [97]:
# Definimos una función que nos cuente las victorias segun los Cavs sean locales o visitantes, para luego mapear el
# dataset filtrado:
def count_victorias(x):
    victorias=0
    if x[2]=="Cleveland Cavaliers":
        if x[3]>x[4]:
            victorias=victorias+1
    if x[5]=="Cleveland Cavaliers":
        if x[3]<x[4]:
            victorias=victorias+1
    return victorias

In [184]:
# Mapeamos con la función y usamos la acción sum() que nos devolverá la suma del contador creado en la función
# ¡ Lo hemos vuelto a hacer!
# El proceso se ha hecho sin utilizar ninguna acción hasta el final. (excepto los take para comprobar los pasos) :
partidos_cavs_enero_sum = partidos_cavs_enero.map(lambda x : count_victorias(x)).sum()
partidos_cavs_enero_sum

83

### Respuesta: 

In [188]:
print(" Solución 7: Los Cavs han ganado durante el mes de Enero %s partidos, ya sea como local o visitante."  % (partidos_cavs_enero_sum))

 Solución 7: Los Cavs han ganado durante el mes de Enero 83 partidos, ya sea como local o visitante.


## Octava pregunta: ¿Los Warrios son mejores fuera de casa o en casa? Golden State Warriors

 Recordar que al nombrar los elementos los hemos ordenado alfabéticamente y ahora los locales están en posición delante de los visitantes al contrario que el dataset original. Entende mos como ser mejores el ganar el partido.

In [134]:
# 1º Filtramos los partidos de Golden State Warriors.
partidos_warriors = partidos_sep.filter(lambda x : x[2] == "Golden State Warriors" or x[5] == "Golden State Warriors")
partidos_warriors.take(5)

[Row(Date=datetime.datetime(2007, 10, 30, 0, 0), Dstart='"7:30 pm"', Home='Golden State Warriors', PTShom=96, PTSvis=117, Visitor='Utah Jazz'),
 Row(Date=datetime.datetime(2007, 11, 2, 0, 0), Dstart='"7:30 pm"', Home='Los Angeles Clippers', PTShom=120, PTSvis=114, Visitor='Golden State Warriors'),
 Row(Date=datetime.datetime(2007, 11, 3, 0, 0), Dstart='"7:00 pm"', Home='Utah Jazz', PTShom=133, PTSvis=110, Visitor='Golden State Warriors'),
 Row(Date=datetime.datetime(2007, 11, 6, 0, 0), Dstart='"7:30 pm"', Home='Golden State Warriors', PTShom=104, PTSvis=108, Visitor='Cleveland Cavaliers'),
 Row(Date=datetime.datetime(2007, 11, 8, 0, 0), Dstart='"7:30 pm"', Home='Golden State Warriors', PTShom=115, PTSvis=120, Visitor='Dallas Mavericks')]

In [138]:
# 2ºComo buen Data Scientis, reutilizamos código de la función anterior, ahora para definir una función que nos acumule
# por separado cuando los Warriors ganan como local y cuando de visitante, para tener una pareja de valores que sean las victorias
# como locales y como visitantes.
def count_victorias_casaovisit(x):
    victorias_local=0
    victorias_visitante=0
    if x[2]=="Golden State Warriors":
        if x[3]>x[4]:
            victorias_local=victorias_local+1
    if x[5]=="Golden State Warriors":
        if x[3]<x[4]:
            victorias_visitante=victorias_visitante+1
    return (victorias_local, victorias_visitante)

In [149]:
partidos_warriors_ganados = partidos_warriors.map(lambda x : count_victorias_casaovisit(x))
partidos_warriors_ganados.take(10)

[(0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (0, 0),
 (1, 0),
 (0, 1),
 (0, 1),
 (0, 0)]

In [155]:
# 3º Hacemos un reduce sumando las dos columnas.  Recordar que al nombrar los elementos los hemos ordenado 
# alfabéticamente y ahora los locales están en posición delante de los visitantes al contrario que el dataset original.
partidos_warriors_ganados_total = partidos_warriors_ganados.reduce(lambda x,y: (x[0]+y[0],x[1]+y[1]))
partidos_warriors_ganados_totalpartidos_warriors_ganados_total

(308, 215)

### Respuesta :

In [159]:
print(" Solución 8: Los Warriors históricamente han sido mejores en CASA al ganar %s partidos, frente a  %s de victorias como visitantes."  % ((partidos_warriors_ganados_total[0]),partidos_warriors_ganados_total[1]))

 Solución 8: Los Warriors históricamente han sido mejores en CASA al ganar 308 partidos, frente a  215 de victorias como visitantes.


## Novena pregunta: Equipo que ha quedado primerio en victorias más temporadas. (si es que hay alguno que más) Buscar el equipo que mas temporadas ha sido el que mas victorias ha tenido. El equipo que mas años mas victorias ha tenido (por año)


In [ ]:
# Mi idea es mapear por clave el año y valor en nombre de los equipos locales y visitantes y sus puntos.
# Luego mapear de nuevo con una funcion que cuente si ha ganado el equipo local o visitante se quede solo con el nombre del equipo
# ganador y un contador.
# Luego agrupar por temporada teniendo el nombre de los equipos y un contador con sus victorias
# Por ultimo ordenarlos de mayor a menor y quedarte con el primero.

In [234]:
p = partidos_sep.map(lambda x: ((x[0].year),(x[2],x[5],x[3],x[4])))
p.take(5)

[(2007, ('Golden State Warriors', 'Utah Jazz', 96, 117)),
 (2007, ('Los Angeles Lakers', 'Houston Rockets', 93, 95)),
 (2007, ('San Antonio Spurs', 'Portland Trail Blazers', 106, 97)),
 (2007, ('Cleveland Cavaliers', 'Dallas Mavericks', 74, 92)),
 (2007, ('Denver Nuggets', 'Seattle SuperSonics', 120, 103))]

In [225]:
def victoria(x):
    if x[1][2]> x[1][3]:
        return x[1][0]
    if x[1][2]< x[1][3]:
        return x[1][1]


In [245]:
# Ahora tenemos la clave temporada equipo y un contador de victorias.
pa = p.map(lambda x: ((x[0],victoria(x)),1))
pa.take(5)

[((2007, 'Utah Jazz'), 1),
 ((2007, 'Houston Rockets'), 1),
 ((2007, 'San Antonio Spurs'), 1),
 ((2007, 'Dallas Mavericks'), 1),
 ((2007, 'Denver Nuggets'), 1)]

In [247]:
# Ahora tenemos todas las victorias de todos los equipos cada temporada
pa.reduceByKey(lambda x,y : x+y).take(10)

[((2008, 'Washington Wizards'), 36),
 ((2007, 'New York Knicks'), 8),
 ((2008, 'New York Knicks'), 27),
 ((2016, 'Golden State Warriors'), 87),
 ((2017, 'Detroit Pistons'), 22),
 ((2014, 'Charlotte Bobcats'), 29),
 ((2012, 'Dallas Mavericks'), 47),
 ((2010, 'Minnesota Timberwolves'), 16),
 ((2010, 'Boston Celtics'), 66),
 ((2016, 'Charlotte Hornets'), 53)]

In [271]:
# Hacemos ahora que la clave sea el año
pa2 = pa.reduceByKey(lambda x,y : x+y).map(lambda x : ((x[0][0]), (x[0][1], x[1])))
pa2.take(5)

[(2008, ('Washington Wizards', 36)),
 (2007, ('New York Knicks', 8)),
 (2008, ('New York Knicks', 27)),
 (2016, ('Golden State Warriors', 87)),
 (2017, ('Detroit Pistons', 22))]

In [323]:
# Conseguimos el mayor numero de victorias por temporada, luego joinearemos para recuperar el nombre de los equipos:
pa3 = pa2.map(lambda x : (x[0],x[1][1])).reduceByKey(lambda x,y: max(x,y))
pa3.take(5)


[(2016, 87), (2017, 54), (2007, 26), (2008, 84), (2009, 81)]

In [322]:
# Creo la misma clave para poder hacer el join:
joinear = pa2.map(lambda x : (x[1][1], x[1][0]))

In [328]:
# Ya tengo los resultados, vemos como en temporadas hay equipos con el mismo número de victorias, compartiendo maximo
casi_solucion=pa3.map(lambda x: (x[1], x[0])).join(joinear)
casi_solucion.collect()

[(88, (2015, 'Golden State Warriors')),
 (84, (2008, 'Boston Celtics')),
 (26, (2007, 'New York Knicks')),
 (26, (2007, 'New Orleans Hornets')),
 (26, (2007, 'Indiana Pacers')),
 (26, (2007, 'Orlando Magic')),
 (26, (2007, 'Denver Nuggets')),
 (26, (2007, 'Utah Jazz')),
 (26, (2007, 'Boston Celtics')),
 (82, (2012, 'San Antonio Spurs')),
 (54, (2017, 'Brooklyn Nets')),
 (54, (2017, 'Golden State Warriors')),
 (54, (2017, 'San Antonio Spurs')),
 (54, (2017, 'Los Angeles Clippers')),
 (54, (2017, 'Philadelphia 76ers')),
 (71, (2010, 'Los Angeles Lakers')),
 (81, (2009, 'Los Angeles Lakers')),
 (53, (2011, 'Charlotte Hornets')),
 (53, (2011, 'Denver Nuggets')),
 (53, (2011, 'Denver Nuggets')),
 (53, (2011, 'Indiana Pacers')),
 (53, (2011, 'Chicago Bulls')),
 (73, (2014, 'San Antonio Spurs')),
 (73, (2014, 'San Antonio Spurs')),
 (87, (2016, 'Golden State Warriors')),
 (85, (2013, 'Miami Heat'))]

In [336]:
casi_solucion.map(lambda x: ((x[1][1]),1) ).reduceByKey(lambda x, y: x+y).sortBy(lambda x: x[1], ascending=False).take(1)

[('San Antonio Spurs', 4)]

VEMOS QUE SON LOS SPURS!

##  Décima pregunta: Escribe la expresión regular correcta que sólo macheen los teléfonos y el correo del siguiente texto.

Si eres cliente y necesitas información sobre tus posiciones, productos o realizar operaciones: Desde España. Desde el extranjero. Banca telefónica en castellano. Bandera castellano. 902 13 23 13. Banca telefónica en catalán. Bandera catalana. 902 88 30 08. Banca telefónica en inglés. Bandera inglesa. 902 88 88 35. O por correo electrónico a atencioncliente@bankinter.com

### Respuesta :


In [ ]:
'^(\d+)|(\w+@\w+.\w+)'

In [ ]:
# Cerramos el Spark Context para no consumir recursos a otros compañeros de cluster ;) 
sc.stop()